In [1]:
#import libraries

import requests
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         220 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.17.0-py_0      conda-forge

The following packages will be UPDATED:

    certifi:       2018.8.24-py36_1 conda-forge --> 2018.8.24-py36_1001 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geog

In [2]:
!pip install beautifulsoup4

distributed 1.21.8 requires msgpack, which is not installed.


In [3]:
!pip install lxml

distributed 1.21.8 requires msgpack, which is not installed.


In [4]:
!pip install html5lib

distributed 1.21.8 requires msgpack, which is not installed.


In [5]:
!pip install requests

distributed 1.21.8 requires msgpack, which is not installed.


In [6]:
from bs4 import BeautifulSoup

In [7]:
#Assign website link to a variable
wikipedia_link='https://www.zipcodestogo.com/New%20Jersey/'

In [8]:
#Use the function get from the requests library to download the Wikipedia page using the wikipedia_link as an argument. Assign the object to the variable postal_code_page.

#hint: requests.get()
source = requests.get(wikipedia_link).text
soup= BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [9]:
table = soup.table
#print(table)
postheader = table.find_all('tr')
#print(postheader)
 

In [10]:
! pip install tabulate

    100% |████████████████████████████████| 51kB 6.7MB/s 
  Running setup.py bdist_wheel for tabulate ... done
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
Successfully built tabulate
distributed 1.21.8 requires msgpack, which is not installed.


Get the data into a dataframe "DATA"

In [11]:
from tabulate import tabulate
table = soup.find_all('table')[1] 
df_list = pd.read_html(str(table))
#df_list[0]
type(df_list)
#print( tabulate(df_list[0], headers='keys', tablefmt='psql') )

data = pd.DataFrame(df_list[0])
type(data)
#data # 290*3

new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

type(data) #289*3
len(data)
data.head()

,Zip Codes for the State of New Jersey,nan,nan,nan
1,Zip Code,City,County,Zip Code Map
2,07001,Avenel,Middlesex,View Map
3,07002,Bayonne,Hudson,View Map
4,07003,Bloomfield,Essex,View Map
5,07004,Fairfield,Essex,View Map


Reassign the index

In [13]:
data = data.reset_index(drop=True)
len(data)
data.head()

,Zip Codes for the State of New Jersey,nan,nan,nan
0,Zip Code,City,County,Zip Code Map
1,07001,Avenel,Middlesex,View Map
2,07002,Bayonne,Hudson,View Map
3,07003,Bloomfield,Essex,View Map
4,07004,Fairfield,Essex,View Map


In [14]:
data.columns = data.iloc[0]
#data = data.reindex(data.index.drop(0)).reset_index(drop=True)
#data.columns.name = None
data.head()


,Zip Code,City,County,Zip Code Map
0,Zip Code,City,County,Zip Code Map
1,07001,Avenel,Middlesex,View Map
2,07002,Bayonne,Hudson,View Map
3,07003,Bloomfield,Essex,View Map
4,07004,Fairfield,Essex,View Map


Drop the column Zip Code as it is not needed

In [15]:
data_1 = data.drop('Zip Code', axis=1)



Drop the column Zip Code Map as it is not needed

In [16]:
data_2 = data_1.drop('Zip Code Map', axis=1)

Drop null values if any from the data frame

In [17]:
data_2.dropna()

,City,County
0,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex
5,Boonton,Morris
6,Caldwell,Essex
7,Caldwell,Essex
8,Carteret,Middlesex
9,Cedar Grove,Essex


In [19]:
data_2.head()
len(data_2)
data_2.head()

,City,County
0,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex


Make the first row as the column header

In [20]:
data_2 = data_2.iloc[1:]
data_2.head()

,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex
5,Boonton,Morris


Drop duplicates on the basis of City & County fields

In [21]:
data_2 = data_2.drop_duplicates(subset=['City','County'])
data_2.head()
len(data_2)

584

Drop row with City values 'Mc Afee'  & 'Sussex' as it doesn't have geo coordinates

In [28]:
data_2 = data_2[data_2.City != 'Mc Afee']

In [29]:
len(data_2)

582

In [30]:
data_2 = data_2[data_2.County != 'Sussex' ]

Drop rows with City value of 'Short Hills' as it doesn't return nearby venues from FourSquare

In [31]:
data_2 = data_2[data_2.City != 'Short Hills']

In [32]:
data_2 = data_2.reset_index(drop=True)

In [33]:
print(len(data_2))
data_2.head()

557


,City,County
0,Avenel,Middlesex
1,Bayonne,Hudson
2,Bloomfield,Essex
3,Fairfield,Essex
4,Boonton,Morris


In [34]:
len(data_2)

557

In [35]:
data_2.head()

,City,County
0,Avenel,Middlesex
1,Bayonne,Hudson
2,Bloomfield,Essex
3,Fairfield,Essex
4,Boonton,Morris


In [36]:
column_names = ['Latitude', 'Longitude'] 
n_hood = pd.DataFrame(columns=column_names)
n_hood.shape

(0, 2)

In [37]:
for index, row in data_2.iterrows():
    try:
        address_1 = row['City'] 
        address = address_1+","+row['County']+","+"USA"
        #print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(row['District'],address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        n_hood
        pass
    except ValueError as error_message:
        print("Error")
        pass
    except AttributeError:
        #print("Problem with data or cannot Geocode.")
        address = row['County']+","+"USA"
        print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        #print(row['District'],address, latitude, longitude)
        n_hood
        pass


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Somerset,USA


Concatenate the latitude and longitude values to the main dataframe to complete the results

In [38]:
len(n_hood)

557

In [41]:
df = pd.concat([data_2, n_hood[['Latitude', 'Longitude']]], axis=1)
df.shape

(557, 4)

In [42]:
df.head()

,City,County,Latitude,Longitude
0,Avenel,Middlesex,40.580382,-74.285147
1,Bayonne,Hudson,40.668714,-74.114309
2,Bloomfield,Essex,40.806767,-74.185423
3,Fairfield,Essex,40.883710,-74.305982
4,Boonton,Morris,40.902599,-74.407097


In [43]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['County'].unique()),
        df.shape[0]
    )
)

The dataframe has 20 boroughs and 557 neighborhoods.


In [44]:
#address = 'New York City, NY'
#address = 'Scarborough,Rouge,Malvern,Ontario,Canada'
address = 'New Jersey,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.0757384, -74.4041622.


EXPLORE New Jersey

In [45]:
# create map of New York using latitude and longitude values
map_nj = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['County'], df['City']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nj) 
    
map_nj


n_hood_1 = df
n_hood_1.head()
n_hood_2 = n_hood_1.drop('Postcode', axis=1)
len(n_hood_2)
n_hood_2.head()

In [47]:
essex_data = df[df['County'] == 'Essex'].reset_index(drop=True)
len(essex_data)
essex_data.head()

,City,County,Latitude,Longitude
0,Bloomfield,Essex,40.806767,-74.185423
1,Fairfield,Essex,40.883710,-74.305982
2,Caldwell,Essex,40.839822,-74.276537
3,Cedar Grove,Essex,40.851766,-74.229035
4,East Orange,Essex,40.767323,-74.204868


In [48]:
address = 'Essex, New Jersey'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7913922, -74.2479444.


In [49]:
# create map of Manhattan using latitude and longitude values
map_essex = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(essex_data['Latitude'], essex_data['Longitude'], essex_data['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_essex)  
    
map_essex

In [50]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180924' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-ID
CLIENT_SECRET:your-client-secret


In [51]:
#address = 'New York City, NY'
#address = 'Scarborough,Rouge,Malvern,Ontario,Canada'
address = 'New Jersey,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


name = 'Essex' # neighborhood name
print(name)

print('Latitude and longitude values of {} are {}, {}.'.format(name, 
                                                               latitude, 
                                                               longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.0757384, -74.4041622.
Essex
Latitude and longitude values of Essex are 40.0757384, -74.4041622.


In [52]:
essex_data.loc[16, 'City']

'Newark'

In [53]:
neighborhood_latitude = essex_data.loc[16, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = essex_data.loc[16, 'Longitude'] # neighborhood longitude value

neighborhood_name = essex_data.loc[16, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Newark are 40.735657, -74.1723667.


In [54]:
# type your answer here
# type your answer here
CLIENT_ID = 'EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW'
CLIENT_SECRET = '3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1'
VERSION = '20180922'
radius = 1000
LIMIT = 500

#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW&client_secret=3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1&v=20180922&ll=40.735657,-74.1723667&radius=1000&limit=500'

In [55]:
results = requests.get(url).json()
#results

In [56]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues


In [58]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    #print(nearby_venues)
    return(nearby_venues)

In [60]:
essex_venues = getNearbyVenues(names=essex_data['City'],
                                   latitudes=essex_data['Latitude'],
                                   longitudes=essex_data['Longitude']
                                  )


Bloomfield
Fairfield
Caldwell
Cedar Grove
East Orange
Essex Fells
Glen Ridge
Livingston
Maplewood
Millburn
Montclair
Verona
Orange
West Orange
Roseland
South Orange
Newark
Belleville
Nutley
Irvington


In [61]:
essex_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belleville,25,25,25,25,25,25
Bloomfield,3,3,3,3,3,3
Caldwell,40,40,40,40,40,40
Cedar Grove,24,24,24,24,24,24
East Orange,14,14,14,14,14,14
Essex Fells,1,1,1,1,1,1
Fairfield,2,2,2,2,2,2
Glen Ridge,16,16,16,16,16,16
Irvington,6,6,6,6,6,6


In [62]:
print('There are {} uniques categories.'.format(len(essex_venues['Venue Category'].unique())))

There are 136 uniques categories.


In [63]:
len(essex_data)

20

In [64]:
# one hot encoding
essex_onehot = pd.get_dummies(essex_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
essex_onehot['Neighborhood'] = essex_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [essex_onehot.columns[-1]] + list(essex_onehot.columns[:-1])
essex_onehot = essex_onehot[fixed_columns]

len(essex_onehot)

426

In [65]:
essex_onehot.shape

(426, 137)

In [66]:
essex_grouped = essex_onehot.groupby('Neighborhood').mean().reset_index()
len(essex_grouped)

20

In [67]:
essex_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Garden,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Colombian Restaurant,Convenience Store,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hockey Arena,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Non-Profit,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Belleville,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.080000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.040000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00000,0.080000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.080000,0.000000,0.040000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.080000,0.000000,0.000000,0.00,0.00000,0.040000,0.000000,0.000000,0.0,0.04000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.040000,0.080000,0.00,0.000,0.000000,0.000000,0.040000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.04,0.000000,0.00,0.000000,0.000000
1,Bloomfield,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000

In [68]:
num_top_venues = 5

for hood in essex_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = essex_grouped[essex_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Belleville----
                venue  freq
0      Sandwich Place  0.08
1          Bagel Shop  0.08
2      Cosmetics Shop  0.08
3  Italian Restaurant  0.08
4   Convenience Store  0.08


----Bloomfield----
              venue  freq
0  Business Service  0.33
1               Gym  0.33
2    Cosmetics Shop  0.33
3        Playground  0.00
4        Non-Profit  0.00


----Caldwell----
                venue  freq
0  Italian Restaurant  0.12
1         Pizza Place  0.10
2  Athletics & Sports  0.08
3                 Spa  0.05
4  Chinese Restaurant  0.05


----Cedar Grove----
                venue  freq
0        Liquor Store  0.08
1  Italian Restaurant  0.08
2                Bank  0.08
3    Business Service  0.04
4       Jewelry Store  0.04


----East Orange----
                 venue  freq
0   Chinese Restaurant  0.21
1  American Restaurant  0.14
2         Liquor Store  0.14
3           Campground  0.07
4        Moving Target  0.07


----Essex Fells----
                 venue  freq
0           

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = essex_grouped['Neighborhood']

for ind in np.arange(essex_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(essex_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
len(neighborhoods_venues_sorted)

20

In [71]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belleville,Italian Restaurant,Bagel Shop,Grocery Store,Convenience Store,Sandwich Place,Cosmetics Shop,Donut Shop,Supplement Shop,Video Store,Shoe Store
1,Bloomfield,Business Service,Cosmetics Shop,Gym,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food
2,Caldwell,Italian Restaurant,Pizza Place,Athletics & Sports,Spa,Chinese Restaurant,Diner,Clothing Store,Coffee Shop,Ramen Restaurant,Cosmetics Shop
3,Cedar Grove,Italian Restaurant,Bank,Liquor Store,Food & Drink Shop,Salon / Barbershop,Pizza Place,Nail Salon,Jewelry Store,Ice Cream Shop,Chinese Restaurant
4,East Orange,Chinese Restaurant,American Restaurant,Liquor Store,Moving Target,Deli / Bodega,Campground,Sandwich Place,Fish & Chips Shop,Furniture / Home Store,Seafood Restaurant
5,Essex Fells,Lake,Yoga Studio,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Food,Fish & Chips Shop
6,Fairfield,Art Gallery,Historic Site,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop
7,Glen Ridge,Gastropub,Bakery,Mobile Phone Shop,Spa,Burger Joint,Sandwich Place,Convenience Store,Baseball Field,Pharmacy,Health & Beauty Service
8,Irvington,Supermarket,Moving Target,Asian Restaurant,Liquor Store,Chinese Restaurant,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
9,Livingston,Italian Restaurant,Ice Cream Shop,Greek Restaurant,Pizza Place,Snack Place,Shopping Mall,Chinese Restaurant,Coffee Shop,Sandwich Place,Salon / Barbershop


In [72]:
essex_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Garden,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Colombian Restaurant,Convenience Store,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hockey Arena,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Non-Profit,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Belleville,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.080000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.040000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00000,0.080000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.080000,0.000000,0.040000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.080000,0.000000,0.000000,0.00,0.00000,0.040000,0.000000,0.000000,0.0,0.04000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.040000,0.080000,0.00,0.000,0.000000,0.000000,0.040000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.04,0.000000,0.00,0.000000,0.000000
1,Bloomfield,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000

In [73]:
# set number of clusters
kclusters = 5

essex_grouped_clustering = essex_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(essex_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 1, 3, 0, 0, 0], dtype=int32)

In [74]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belleville,Italian Restaurant,Bagel Shop,Grocery Store,Convenience Store,Sandwich Place,Cosmetics Shop,Donut Shop,Supplement Shop,Video Store,Shoe Store
1,Bloomfield,Business Service,Cosmetics Shop,Gym,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food
2,Caldwell,Italian Restaurant,Pizza Place,Athletics & Sports,Spa,Chinese Restaurant,Diner,Clothing Store,Coffee Shop,Ramen Restaurant,Cosmetics Shop
3,Cedar Grove,Italian Restaurant,Bank,Liquor Store,Food & Drink Shop,Salon / Barbershop,Pizza Place,Nail Salon,Jewelry Store,Ice Cream Shop,Chinese Restaurant
4,East Orange,Chinese Restaurant,American Restaurant,Liquor Store,Moving Target,Deli / Bodega,Campground,Sandwich Place,Fish & Chips Shop,Furniture / Home Store,Seafood Restaurant
5,Essex Fells,Lake,Yoga Studio,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Food,Fish & Chips Shop
6,Fairfield,Art Gallery,Historic Site,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop
7,Glen Ridge,Gastropub,Bakery,Mobile Phone Shop,Spa,Burger Joint,Sandwich Place,Convenience Store,Baseball Field,Pharmacy,Health & Beauty Service
8,Irvington,Supermarket,Moving Target,Asian Restaurant,Liquor Store,Chinese Restaurant,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
9,Livingston,Italian Restaurant,Ice Cream Shop,Greek Restaurant,Pizza Place,Snack Place,Shopping Mall,Chinese Restaurant,Coffee Shop,Sandwich Place,Salon / Barbershop


In [75]:
essex_merged = essex_data

# add clustering labels
essex_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
essex_merged = essex_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='City')

essex_merged.head() # check the last columns!

,City,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bloomfield,Essex,40.806767,-74.185423,0,Business Service,Cosmetics Shop,Gym,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food
1,Fairfield,Essex,40.883710,-74.305982,2,Art Gallery,Historic Site,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop
2,Caldwell,Essex,40.839822,-74.276537,0,Italian Restaurant,Pizza Place,Athletics & Sports,Spa,Chinese Restaurant,Diner,Clothing Store,Coffee Shop,Ramen Restaurant,Cosmetics Shop
3,Cedar Grove,Essex,40.851766,-74.229035,0,Italian Restaurant,Bank,Liquor Store,Food & Drink Shop,Salon / Barbershop,Pizza Place,Nail Salon,Jewelry Store,Ice Cream Shop,Chinese Restaurant
4,East Orange,Essex,40.767323,-74.204868,0,Chinese Restaurant,American Restaurant,Liquor Store,Moving Target,Deli / Bodega,Campground,Sandwich Place,Fish & Chips Shop,Furniture / Home Store,Seafood Restaurant


In [76]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(essex_merged['Latitude'], essex_merged['Longitude'], essex_merged['City'], essex_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

 Cluster 1 : Seems to be the most happening place with lots of restaurants, bus facilities, supermarkets etc.,

In [77]:
essex_merged.loc[essex_merged['Cluster Labels'] == 0, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bloomfield,Business Service,Cosmetics Shop,Gym,Yoga Studio,Food & Drink Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food
2,Caldwell,Italian Restaurant,Pizza Place,Athletics & Sports,Spa,Chinese Restaurant,Diner,Clothing Store,Coffee Shop,Ramen Restaurant,Cosmetics Shop
3,Cedar Grove,Italian Restaurant,Bank,Liquor Store,Food & Drink Shop,Salon / Barbershop,Pizza Place,Nail Salon,Jewelry Store,Ice Cream Shop,Chinese Restaurant
4,East Orange,Chinese Restaurant,American Restaurant,Liquor Store,Moving Target,Deli / Bodega,Campground,Sandwich Place,Fish & Chips Shop,Furniture / Home Store,Seafood Restaurant
7,Livingston,Italian Restaurant,Ice Cream Shop,Greek Restaurant,Pizza Place,Snack Place,Shopping Mall,Chinese Restaurant,Coffee Shop,Sandwich Place,Salon / Barbershop
8,Maplewood,Pizza Place,Coffee Shop,Ice Cream Shop,Mexican Restaurant,American Restaurant,Diner,Salad Place,Restaurant,Colombian Restaurant,Pub
9,Millburn,Italian Restaurant,Pizza Place,Coffee Shop,New American Restaurant,Deli / Bodega,French Restaurant,Ice Cream Shop,Gym / Fitness Center,Plaza,Playground
10,Montclair,Park,Event Space,Cosmetics Shop,Café,Chinese Restaurant,Bookstore,Bank,Theater,Train Station,Flower Shop
11,Verona,Deli / Bodega,Pizza Place,Italian Restaurant,Sporting Goods Shop,Arts & Crafts Store,Bus Stop,Convenience Store,Cosmetics Shop,Garden Center,Gift Shop
12,Orange,Bakery,Sandwich Place,Shoe Store,Italian Restaurant,Southern / Soul Food Restaurant,Gym,Latin American Restaurant,Mobile Phone Shop,Pizza Place,Donut Shop


Cluster 2: Must be a place with a lake to relax and have some food

In [78]:
essex_merged.loc[essex_merged['Cluster Labels'] == 1, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Essex Fells,Lake,Yoga Studio,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Food,Fish & Chips Shop


Clutser 3: Good for a day's outing to see the old world charm

In [79]:
essex_merged.loc[essex_merged['Cluster Labels'] == 2, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Fairfield,Art Gallery,Historic Site,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop


Cluster 4: Must be a place for pub and baseball enthusiasts to hang around

In [80]:
essex_merged.loc[essex_merged['Cluster Labels'] == 3, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Glen Ridge,Gastropub,Bakery,Mobile Phone Shop,Spa,Burger Joint,Sandwich Place,Convenience Store,Baseball Field,Pharmacy,Health & Beauty Service


Cluster 5: Place for shopping and eatout

In [81]:
essex_merged.loc[essex_merged['Cluster Labels'] == 4, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Newark,Pizza Place,Deli / Bodega,Bar,Donut Shop,Shoe Store,Coffee Shop,American Restaurant,Bank,Southern / Soul Food Restaurant,Burger Joint
